## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-17-28-18 +0000,nyt,Trump Promises to Help Europe Speed Weapons to...,https://www.nytimes.com/2025/07/14/us/politics...
1,2025-07-14-17-26-56 +0000,cbc,U.S. helicopter maker settles with families of...,https://www.cbc.ca/news/politics/sikorsky-airc...
2,2025-07-14-17-17-56 +0000,nyt,How Trump Changed His Tone on Putin and the Ru...,https://www.nytimes.com/2025/07/14/us/politics...
3,2025-07-14-17-11-33 +0000,bbc,MP charged for accusing Lesotho king of signin...,https://www.bbc.com/news/articles/cp86pznd65ko
4,2025-07-14-17-05-04 +0000,bbc,Nine killed and dozens hurt in fire at Massach...,https://www.bbc.com/news/articles/cq6mp9ydl73o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,12
14,killed,5
6,ukraine,5
65,world,3
140,tariff,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
21,2025-07-14-14-18-31 +0000,nyt,Trump Reluctantly Comes Around to Backing Ukra...,https://www.nytimes.com/2025/07/14/us/politics...,28
48,2025-07-13-23-03-47 +0000,nyt,King Charles III Will Host Trump for U.K. Stat...,https://www.nytimes.com/2025/07/13/world/europ...,26
0,2025-07-14-17-28-18 +0000,nyt,Trump Promises to Help Europe Speed Weapons to...,https://www.nytimes.com/2025/07/14/us/politics...,25
32,2025-07-14-11-20-22 +0000,nyt,Trump Is Expected to Announce New Weapons Pipe...,https://www.nytimes.com/2025/07/14/world/europ...,25
30,2025-07-14-12-07-37 +0000,bbc,Trump will be hosted by King at Windsor during...,https://www.bbc.com/news/articles/c4g25ne7gw6o,24


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
21,28,2025-07-14-14-18-31 +0000,nyt,Trump Reluctantly Comes Around to Backing Ukra...,https://www.nytimes.com/2025/07/14/us/politics...
4,16,2025-07-14-17-05-04 +0000,bbc,Nine killed and dozens hurt in fire at Massach...,https://www.bbc.com/news/articles/cq6mp9ydl73o
5,13,2025-07-14-17-03-25 +0000,nyt,The Economy Has Been Resilient. The New Round ...,https://www.nytimes.com/2025/07/14/us/politics...
3,12,2025-07-14-17-11-33 +0000,bbc,MP charged for accusing Lesotho king of signin...,https://www.bbc.com/news/articles/cp86pznd65ko
39,12,2025-07-14-08-01-26 +0000,bbc,Don't vilify Air India crash crew: Indian pilo...,https://www.bbc.com/news/articles/cddzp0g5l25o
13,11,2025-07-14-15-51-39 +0000,bbc,Nigeria declares public holiday to honour form...,https://www.bbc.com/news/articles/c93kxl08glwo
25,11,2025-07-14-13-09-52 +0000,nyt,Companies in Europe Are Frozen as Tariff Uncer...,https://www.nytimes.com/2025/07/14/business/ta...
14,10,2025-07-14-15-48-36 +0000,nyt,BBC Ends Ties With ‘MasterChef’ Host Gregg Wal...,https://www.nytimes.com/2025/07/14/world/europ...
8,10,2025-07-14-16-48-00 +0000,nyt,"Maya Ruler’s Tomb Is Unearthed in Belize, With...",https://www.nytimes.com/2025/07/10/world/ameri...
43,10,2025-07-14-05-00-05 +0000,bbc,Nigerian woman smuggled baby into UK using fak...,https://www.bbc.com/news/articles/c98jl8jnz92o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
